In [2]:
import os 

In [3]:
%pwd

'c:\\Users\\admin\\Desktop\\Predictive Maintenance\\Predictive_Maintenance_With_MLops\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\admin\\Desktop\\Predictive Maintenance\\Predictive_Maintenance_With_MLops'

In [6]:
# entity 

from dataclasses import dataclass 
from pathlib import Path 



@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path 
    train_data_path: Path 
    test_data_path: Path
    model_name: str
    imputer_name: str
    alpha: float
    l1_ratio: float
    target_column: str  

    
    

In [7]:
from src.Mlflow_Project.constants import *
from src.Mlflow_Project.utils.utility import FileOperations

In [8]:
# configuration
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = FileOperations.read_yaml(config_filepath)
        self.params = FileOperations.read_yaml(params_filepath)
        self.schema = FileOperations.read_yaml(schema_filepath)

        FileOperations.create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN


        FileOperations.create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            imputer_name= config.imputer_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name

        )

        return model_trainer_config

In [9]:
# defining component

import pandas as pd
import os 
from src.Mlflow_Project.__init__ import logger 
from sklearn.linear_model import ElasticNet
from sklearn.impute import SimpleImputer
import joblib 

In [22]:
# components 

class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Drop rows with missing values in the target column
        train_data.dropna(subset=[self.config.target_column], inplace=True)
        test_data.dropna(subset=[self.config.target_column], inplace=True)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]

        # Create an imputer to fill missing values with a specific strategy (e.g., mean or median)
        imputer = SimpleImputer(strategy='mean')

        # Fit the imputer on the training data and transform the test data
        train_y_impute = imputer.fit_transform(train_y)
        test_y = imputer.transform(test_data[[self.config.target_column]].values.reshape(-1, 1))

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y_impute)

        test_x = pd.DataFrame(test_x, columns=test_x.columns)
        test_y = pd.DataFrame(test_y, columns=[self.config.target_column])

        # Save the scaled datasets and target variables
        test_x.to_csv(os.path.join(self.config.root_dir, "test_x.csv"), index=False)
        test_y.to_csv(os.path.join(self.config.root_dir, "test_y.csv"), index=False)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        joblib.dump(imputer, os.path.join(self.config.root_dir, self.config.imputer_name))


In [23]:
# Defining Pipeline 

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config = model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    print("An error occurred:", e)

[2023-10-31 14:05:58,064: INFO: yaml file: config\config.yaml loaded successfully]
[2023-10-31 14:05:58,069: INFO: yaml file: params.yaml loaded successfully]
[2023-10-31 14:05:58,079: INFO: yaml file: schema.yaml loaded successfully]
[2023-10-31 14:05:58,082: INFO: created directory at: artifacts]
[2023-10-31 14:05:58,085: INFO: created directory at: artifacts/model_trainer]


c:\Users\admin\Desktop\Predictive Maintenance\Predictive_Maintenance_With_MLops\.venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
